# Importing all the libraries

In [ ]:
import pandas as pd
import math

#plotting
import matplotlib.pyplot as plt

import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
#import sklearn.externals.joblib as extjoblib
import joblib
import time

#Bayesian optimizer

import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.plots import plot_histogram, plot_objective_2D
from skopt.utils import use_named_args


# ANN
#Sequential for initializing the neural network
#Dense for adding a densely connected neural network layer
#LSTM for adding the Long Short-Term Memory layer
#Dropout for adding dropout layers that prevent overfitting
from numpy import array
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LeakyReLU

from keras import backend as K
from keras.callbacks import History
from keras import optimizers
from keras.layers import Dropout
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard

import os
os.environ['PYTHONHASHSEED'] = '0'
import tensorflow as tf
import random as rn

# Warnings
import warnings
warnings.filterwarnings("ignore")

# File Paths and version name

In [ ]:
keras_model_path=r"C:\Users\Keras-Model\\"
database_path=r"C:\Users\Database\\"
Result_path=r"C:\Users\Results\\"
version="49v4" 

# Version of databased used for training and validation of ANN
output="FOPR" 
scalerX_name="scalerX_"+output
scalerY_name="scalerY_"+output
path_best_model_fwir =  r"C:\Users\Keras-Model\fwir_Olymp_"+version+"_minmax.keras"
path_best_model_fopr =  r"C:\Users\Keras-Model\fopr_Olymp_"+version+"_minmax.keras"
path_best_model_fwpr =  r"C:\Users\Keras-Model\fwpr_Olymp_"+version+"_minmax.keras"

# Load Dataset

In [ ]:
# load dataset
data = pd.read_csv(r"C:\Users\Database\Olympus_"+version+".csv",sep=',')
#dataset = dataframe.values
data.shape
data.head()



# Defining ANN model architecture and Data Splitting
The keras API has two modes of consturcting neural networks. The simplest is the sequential model which only allows for the layers to be added in sequence.

In [ ]:
#Builing ANN Model
def ANN_model(X_train, Y_train, learning_rate, dropout, n_units, n_layers):
    
   #Reset the network graphs for results repdroucability in Keras
    N=17 #seed number to give a starting point to random number generate
    np.random.seed(N)
    rn.seed(N)
    session_conf=tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
    tf.random.set_seed(N)
    K.clear_session()
    sess=tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.get_session()
    
    #start construction of the sequential model
    model=Sequential()
    if n_layers ==1:
        #add a input layer
        
        model.add(Dense(n_units, activation='relu', input_dim=X_train.shape[1]))
    else:
        model.add(Dense(n_units, activation='relu', input_dim=X_train.shape[1]))
        for i in range(n_layers-1):
            model.add(Dense(n_units, activation='relu')) 
            #ReLU standsfor rectified linear unit, and is a type of activation function. Mathematically, it is defined as y = max(0, x). Visually, it looks like the following: 
         #ReLU is the most commonly used activation function in neural networks, especially in CNN
    #add dropout
    model.add(Dropout(dropout))
    #Add dense layer
    model.add(Dense(Y_train.shape[1], activation='linear'))
    #model.add(LeakyRelU(alpha=0.1))
    
# Neural network has been defined and must be finalized by adding a loss_function, optimizer and performance metrics
#This is called "compilation" in Keras. One can either define the optimizer using string or if wants to have more control 
# of its parameters then instantiate an object, like learning_rate in this case.

    adam=optimizers.Adam(lr=learning_rate)
    # compiling ANN model using adam optimizer and mse loss function
    #model.compile(optimizer=adam, loss='mse',metrics=['accuracy','mean_absolute_percentage_error','root_mean_squared_error'])
    model.compile(optimizer=adam, loss='mae',metrics=['mae','mse'])
    return model


def data_split(data, feature_list, target, lifetime ):
    """ 
    Splits data into train, validation and test sets 
    
    Parameters
    ----------
    data: Dataframe, shape=[n_points, n_features]
    All feature data
    feature list : list
        list of features to include into training
    target: list
        List of target variables to include into training
    Returns
    ------
    -Train, validation and test sets, array-like
    -scaling functions for X and Y
    """
    X=data[feature_list]
    Y=data[target]
    
    #lifetime is used for entire production cycle of reservoir 
    split=(X.shape[0])/lifetime
    #train_size = int(0.75*X.shape[0])
    #test_size = int(0.90*X.shape[0])
    train_size = int(0.75*split)*lifetime
    test_size = int(0.90*split)*lifetime
    #train_size = 504
    #test_size = 525
    n_target = int(len(target)) #number of target outputs # in this case it give 1 as output
    
    #splitting
#reshape (-1,1) provides column as 1 but rows as unknown , so output will be an array with 1 column and rows as present    
    
    (X_train, Y_train) =(np.array(X[:train_size]), np.array(Y[:train_size]).reshape(-1, n_target))
    (X_val, Y_val) =(np.array(X[train_size:test_size]), np.array(Y[train_size:test_size]).reshape(-1, n_target))
    (X_test, Y_test) =(np.array(X[test_size:]), np.array(Y[test_size:]).reshape(-1, n_target))
    
    #scaling 
    
    scalerX=MinMaxScaler().fit(X_train)
    scalerY=MinMaxScaler().fit(Y_train)
    joblib.dump(scalerX, keras_model_path+ scalerX_name+version+'.gz')
    joblib.dump(scalerY, keras_model_path+ scalerY_name +version+'.gz')
    X_train =scalerX.transform(X_train)
    Y_train=scalerY.transform(Y_train)
    X_val =scalerX.transform(X_val)
    Y_val=scalerY.transform(Y_val)
    X_test =scalerX.transform(X_test)
    Y_test=scalerY.transform(Y_test)
    
    return X_train, Y_train, X_val, Y_val, X_test, Y_test, scalerX, scalerY, train_size, test_size




In [ ]:
[X_train, Y_train, X_val, Y_val, X_test, Y_test, scalerX_name, scalerY_name, train_size, test_size]= \
data_split(data=data, feature_list=["timestep", "BHPI1", "BHPI2", "BHPI3", "BHPI4", "BHPI5", \
                                    "BHPI6", "BHPP1", "BHPP10", "BHPP2", "BHPP3","BHPP4", "BHPP5", \
                                    "BHPP6", "BHPP7","BHPP8", "BHPP9"], target=[output], lifetime=20)

# Plotting Functions

In [ ]:
import math
import matplotlib.pyplot as plt
import statistics
from sklearn.metrics import r2_score


# Error distribution and proxy performance function for test_Data

def plot_train_performance(model,X_train, Y_train, X_val,Y_val,scalerY):
    # model = the ANN model (fitted)
    # X_train = input for the ANN from training database
    # Y_train = output of the training database
    # X_val = input for the ANN from validation database
    # Y_val = output of the validation database
    # lims = 2 points [min,max] for the plot boundary

    # Calculate test data with the model
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_val)
    train_preds=scalerY.inverse_transform(train_preds)
    val_preds=scalerY.inverse_transform(val_preds)
    Y_train=scalerY.inverse_transform(Y_train)
    Y_val=scalerY.inverse_transform(Y_val)
    # Checking performance of test data
    plt.figure(4)
    plt.axes(aspect='equal')
    plt.scatter(Y_train, train_preds, label='Training')
    plt.scatter(Y_val, val_preds, label='Validation', marker='x')
    plt.legend()
    plt.xlabel('True Values,' + output)
    plt.ylabel('Predictions,' + output)
    lims=[np.min(Y_val),np.max(Y_val)]
    plt.xlim(lims)
    plt.ylim(lims)
    _ = plt.plot(lims, lims)
    
    # Getting error distribution
    plt.figure(5)
    error_train = train_preds - Y_train
    error_val = val_preds - Y_val
    plt.hist(error_train, bins=25,label='Training')
    plt.hist(error_val, bins=25,label='Validation')
    plt.legend()
    plt.xlabel('Prediction Error,'+ output +'[Sm3]')
    _ = plt.ylabel('Count')
    
    # Calculate R^2
    r2_train = r2_score(Y_train, train_preds)
    r2_val = r2_score(Y_val, val_preds)
    print("R-square Training: {:.2f}".format(r2_train))
    print("R-square Validation: {:.2f}".format(r2_val))
    ape_train = (abs(Y_train-train_preds))/(Y_train)*100
    max_ape_train = max(ape_train)
  
    mean_ape_train = statistics.mean(ape_train.flatten())
    min_ape_train = min(ape_train)
    print("""Training Average Percentage Error:
    - Maximum=%.2f
    - Mean=%.2f
    - Minimum=%.2f""" % (max_ape_train, mean_ape_train, min_ape_train))
    
    # Calculate Validation Absolute Percentage Error (APE)
    ape_val = (abs(Y_val-val_preds))/(Y_val)*100
    max_ape_val = max(ape_val)
    mean_ape_val = statistics.mean(ape_val.flatten())
    min_ape_val = min(ape_val)
    plt.figure(9)
    plt.hist(ape_train,bins=25,label='APE Training')
    plt.hist(ape_val,bins=25,label='APE Validation')
    plt.legend()
    plt.xlabel('Absolute Percentage Error,'+ output + '[%]')
    _ = plt.ylabel('Count')
    
    print("""Validation Average Percentage Error:
    - Maximum=%.2f
    - Mean=%.2f
    - Minimum=%.2f""" % (max_ape_val, mean_ape_val, min_ape_val))
    
    print ("Train NRMSE is", np.sqrt(mse(Y_train, train_preds))/np.mean(Y_train)*100)
    print ("Val NRMSE is", np.sqrt(mse(Y_val, val_preds))/np.mean(Y_val)*100)
    
def plot_test_performance(model, X_test, Y_test, scalerY):
    # model = the ANN model (fitted)
    # X_test = input for the ANN from test database
    # Y_test = output of the test database
    # lims = 2 points [min,max] for the plot boundary

    # Calculate test data with the model
    test_preds = model.predict(X_test)
    test_preds= scalerY.inverse_transform(test_preds)
    Y_test=scalerY.inverse_transform(Y_test)
    # Checking performance of test data
    plt.figure(6)
    plt.axes(aspect='equal')
    plt.scatter(Y_test, test_preds)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.legend(['Testing'])
    lims=[np.min(test_preds),np.max(test_preds)]
    plt.xlim(lims)
    plt.ylim(lims)
    _ = plt.plot(lims, lims)
    
    # Getting error distribution
    plt.figure(7)
    error = test_preds - Y_test
    plt.hist(error, bins=25,label='Testing')
    plt.legend()
    plt.xlabel('Prediction Error,'+ output +'[Sm3]')
    _ = plt.ylabel('Count')
    
    # Calculate R^2
    r2_test = r2_score(Y_test, test_preds)
    print("R-square Test: {:.2f}".format(r2_test))
    
    # Calculate Absolute Percentage Error (APE)
    ape_test = (abs(Y_test-test_preds))/(Y_test)*100
    max_ape_test = max(ape_test)
    mean_ape_test = statistics.mean(ape_test.flatten())
    min_ape_test = min(ape_test)
    plt.figure(10)
    plt.hist(ape_test,bins=25,label='APE Test')
   
    plt.legend()
    plt.xlabel('Absolute Percentage Error,'+ output + '[%]')
    _ = plt.ylabel('Count')
    print("""Average Percentage Error:
    - Maximum=%.2f
    - Mean=%.2f
    - Minimum=%.2f""" % (max_ape_test, mean_ape_test, min_ape_test))    
    print ("Test NRMSE is", np.sqrt(mse(Y_test, test_preds))/np.mean(Y_test)*100)


# Training ANN Model

In [ ]:

start=time.time()
#default hyperparameters

default_parameters = [0.01, 2, 20, 0.1]
num_epochs = 1000

dim_learning_rate =Real(low=1e-3, high=1e-1, prior='log-uniform', name='learning_rate')
dim_n_layers=Integer(low=1, high=5, name='n_layers')
dim_n_units= Integer(low=20, high=100, name='n_units')
dim_dropout = Real(low=0.01, high=0.1, prior='log-uniform', name='dropout')
dimensions = [dim_learning_rate, dim_n_layers, dim_n_units, dim_dropout]

path_best_model=path_best_model_fopr
best_val_loss = 100.0
@use_named_args(dimensions=dimensions)


def fitness_SPM(learning_rate, n_layers, n_units, dropout):
    
    """
 Hyper-parameters:
learning_rate: Learning-rate for the optimizer.
n_layers: Number of dense layers.
 n_units: Number of nodes in each dense layer.
 dropout: Percentage of the nodes retained.
 window: Number of window
 """

    # Print the hyper-parameters
    print('learning rate: {0:.1e}'.format(learning_rate))
    print('num_dense_layers:', n_layers)
    print('num_dense_nodes:', n_units)
    print('dropout:', dropout)


    model= ANN_model(X_train=X_train, Y_train=Y_train, learning_rate=learning_rate,\
                     dropout=dropout, n_units=n_units, n_layers=n_layers)

#use Keras to train the model.

    patience= 20
    es= EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=patience, restore_best_weights=True)
    history= model.fit(X_train, Y_train, verbose=1, epochs=num_epochs, validation_data=(X_val, Y_val), callbacks=[es])

# Get the classification accuracy on the validation-set
#after the last training-epoch.

    if  es.stopped_epoch == 0:
        val_loss =history.history['val_loss'][-1]
       
    else:
        val_loss=history.history['val_loss'][es.stopped_epoch-patience]
    #Print the classification accuracy.
    print()
    print('val_loss:', val_loss)

    print()
    
    
    #save the model if it improves on the best-found performance
    #we use the global keyword so we update the variable outside of this function
    
    global best_val_loss
    
    # If the classification accuracy of the saved model is improved ...
    if val_loss + loss < best_val_loss:
        # Save the new model to harddisk.
        model.save(path_best_model)
    
    # Update the classification accuracy.
        best_val_loss = val_loss
    
    # Delete the Keras model with these hyper-parameters from memory.
    
    del model
    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters
    K.clear_session()
    np.save('history_'+version+output+'.npy', history.history)
    # NOTE: Scikit-optimize does minimization so it tries to
    # find a set of hyper-parameters with the LOWEST fitness-value.
     # Because we are interested in the HIGHEST classification
     # accuracy, we need to negate this number so it can be minimized
    #history=np.load('history_49_fwir.npy',allow_pickle='TRUE').item()
   # convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(history.history)
    # or save to csv: 
    hist_df.to_csv('history_'+version+ output +'.csv')
    
    return val_loss

# run the hyper-paramter optimization.
search_result = gp_minimize(func=fitness_SPM,dimensions=dimensions, acq_func='EI', n_calls=80,x0=default_parameters)

plot_convergence(search_result)
end=time.time()

space = search_result.x #reference to the search-space object
space.point_to_dict(search_result.x) #Then we can use it to create a dict where the hyper-parameters have 
hours, rem=divmod(end-start, 3600)
minutes, seconds =divmod(rem, 60)
print("Time elapsed: ", int(hours), "hours", int(minutes), "minutes", int(seconds), "seconds")

# Print best hyperparameter values
print(space)


# Plotting Figures

In [ ]:
# Error distribution and proxy performance function for test data
# load the established model, comment 

model_fopr=load_model(path_best_model_fopr )
#model_fwpr=load_model(path_best_model_fwpr )
#model_fwir=load_model(path_best_model_fwir )

b=plot_train_performance(model_fopr,X_train,Y_train,X_val,Y_val,scalerY_name)
c=plot_test_performance(model_fopr,X_test,Y_test,scalerY_name)


## Blind Testsing of FOPR model

In [ ]:
blind_data = pd.read_csv(r"C:\Users\Database\BlindTest.csv",sep=',')

X_blind=blind_data[["timestep", "BHPI1", "BHPI2", "BHPI3", "BHPI4", "BHPI5",\
            "BHPI6", "BHPP1", "BHPP10", "BHPP2", "BHPP3","BHPP4", "BHPP5",\
            "BHPP6", "BHPP7", "BHPP8", "BHPP9"]]
Y_blind_fopr=test_data[["FOPR"]]
Y_blind_fopr=np.array(Y_blind_fopr[:]).reshape(-1,1)

# load model and invert predictions
model_fopr=load_model(path_best_model_fopr )
scalerX_fopr=joblib.load( keras_model_path+scalerX_name+'.gz')
scalerY_fopr=joblib.load( keras_model_path+ scalerY_name'+'.gz')

X_blind=scalerX_fopr.transform(X_blind)
Y_blind=scalerY_fopr.transform(Y_blind_fopr)
test_preds_fopr=model_fopr.predict(X_blind)
test_preds_fopr = scalerY_fopr.inverse_transform(test_preds_fopr)

print ("Blind Test NRMSE is", np.sqrt(mse(Y_blind_fopr, test_preds_fopr))/np.mean(Y_blind_fopr)*100)

xdata = np.arange(1,len(Y_blind_fopr)+1,1)
plt.rcParams['figure.figsize'] = (10,8)
plt.rc('axes', titlesize=20) # fontsize of the axes title
plt.rc('axes', labelsize=20) # fontsize of the x and y labels
plt.rc('xtick', labelsize=15) # fontsize of the tick labels
plt.rc('ytick', labelsize=15) # fontsize of the tick  labels
plt.rc('legend', fontsize=18) # legend fontsize
plt.figure(2)
plt.xlabel('Timestep')
plt.ylabel('FOPR [Sm3/day]')
plt.plot(xdata, Y_blind_fopr[:,0], label='Data', color='black', linewidth=3)
plt.plot(xdata, test_preds_fopr[:,0], label='Model', linestyle=':', color='black', linewidth=3)
plt.legend(['Eclipse', 'SPM'], loc='upper right' )
plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
plt.savefig(Result_path+'FOPR Blind Test.pdf')
plt.show()